# Parameter Interfaces in NRPy

## Author: Zach Etienne
### Formatting improvements courtesy Brandon Clark

## Exploring the NRPy parameter system (`params.py`), this notebook shows how to declare, store, and update parameters that control both Python level infrastructure and generated C code. We will walk through defining user facing run time parameters, registering C code parameters, working with arrays, adjusting defaults (including type changes), and using the symbolic handles inside SymPy expressions to emit simple C code.

### Required reading if you are unfamiliar with programming or [computer algebra systems](https://en.wikipedia.org/wiki/Computer_algebra_system). Otherwise, use for reference; you should be able to pick up the syntax as you follow the tutorial.
+ **[Python Tutorial](https://docs.python.org/3/tutorial/index.html)**
+ **[SymPy Tutorial](http://docs.sympy.org/latest/tutorial/intro.html)**

### NRPy Source Code for this module:  
* [params.py](../edit/params.py)

# Table of Contents

The module is organized as follows:

1. [Step 1](#Step-1:-Initialize-core-Python-and-NRPy-parameter-modules): Initialize core Python and NRPy parameter modules
1. [Step 2](#Step-2:-Creating-simple-Python-level-parameters-with-register_param()): Creating simple Python level parameters with register_param()
1. [Step 3](#Step-3:-Looking-up-and-modifying-parameter-values-from-strings): Looking up and modifying parameter values from strings
1. [Step 4](#Step-4:-Exploring-global-parameter-dictionaries-and-descriptions): Exploring global parameter dictionaries and descriptions
1. [Step 5](#Step-5:-CodeParameters-vs-NRPyParameters:-runtime-tuning-for-generated-code): CodeParameters vs NRPyParameters: runtime tuning for generated code
1. [Step 6](#Step-6:-Registering-multiple-CodeParameters-at-once-with-register_CodeParameters()): Registering multiple CodeParameters at once with register_CodeParameters()
1. [Step 7](#Step-7:-Working-with-array-valued-CodeParameters): Working with array valued CodeParameters
1. [Step 8](#Step-8:-Adjusting-CodeParameter-defaults-after-registration): Adjusting CodeParameter defaults after registration
1. [Step 9](#Step-9:-Using-symbolic-CodeParameters-inside-SymPy-expressions-and-emitting-C-code): Using symbolic CodeParameters inside SymPy expressions and emitting C code

# Step 1: Initialize core Python and NRPy parameter modules
### \[Back to [top](#Table-of-Contents)\]

The NRPy parameter system lives in `nrpy.params`. It provides two main families of parameters:

* Python level parameters for infrastructure choices and high level options.
* CodeParameters that will appear as configurable run time parameters in generated C code.

Let us import the module, along with SymPy, and peek at the default Python level parameters that `params.py` registers when imported.

In [ ]:
# Step 1: Initialize core Python and NRPy parameter modules
import nrpy.params as par   # NRPy: parameter interface
import sympy as sp          # SymPy: used for symbolic CodeParameters

print("Default Python level parameters already registered:")
for name in ["Infrastructure", "fp_type", "parallelization", "clang_format_options"]:
    if name in par.glb_params_dict:
        p = par.glb_params_dict[name]
        print(f"  {p.module}::{p.name} = {p.value!r}")

# Step 2: Creating simple Python level parameters with register_param()
### \[Back to [top](#Table-of-Contents)\]

Python level parameters are small objects collected in `par.glb_params_dict`. They are ideal for:

* Choosing backends (for example setting the infrastructure).
* Enabling or disabling features.
* Storing simple numeric settings that only affect Python side logic.

The main helper is:

```python
par.register_param(py_type, module, name, value, description="")
```

* `py_type`: one of `int`, `float`, `bool`, or `str`.
* `module`: a grouping label (often the name of your NRPy module).
* `name`: the parameter name.
* `value`: the default Python value.
* `description`: a short human readable description.

Here we define a small set of tutorial parameters for a 1D wave equation example, all grouped under the module name `"wave_tutorial"`.

In [ ]:
# Step 2: Creating simple Python level parameters with register_param()

par.register_param(
    py_type=int,
    module="wave_tutorial",
    name="grid_points",
    value=200,
    description="Number of spatial grid points for the 1D wave example.",
)

par.register_param(
    py_type=float,
    module="wave_tutorial",
    name="final_time",
    value=50.0,
    description="Final evolution time for the 1D wave example.",
)

par.register_param(
    py_type=bool,
    module="wave_tutorial",
    name="use_reflecting_bc",
    value=True,
    description="Use reflecting boundary conditions at both ends.",
)

print("Tutorial Python level parameters:")
for name in ["grid_points", "final_time", "use_reflecting_bc"]:
    p = par.glb_params_dict[name]
    print(
        f"  {p.module}::{p.name} = {p.value!r} "
        f"(type {p.py_type.__name__}); description = '{p.description}'"
    )

# It is helpful to see what happens if we try an unsupported Python type.
print("\nAttempting to register a parameter with an invalid Python type:")
try:
    par.register_param(
        py_type=list,
        module="wave_tutorial",
        name="bad_param",
        value=[],
        description="This should fail because list is not a valid param type.",
    )
except ValueError as e:
    print("  Caught ValueError:", e)

# Step 3: Looking up and modifying parameter values from strings
### \[Back to [top](#Table-of-Contents)\]

Once parameters are registered, you usually work with them via string based helpers:

* `par.parval_from_str(parameter_name)` returns the current value.
* `par.set_parval_from_str(parameter_name, new_value)` updates the value.

The `parameter_name` can be:

* A bare name like `"grid_points"`, or
* A string with a module prefix like `"wave_tutorial::grid_points"`.

In both cases, only the part after `::` is used to look up the parameter. The module label is stored separately in the parameter object.

Let us query and update a couple of our tutorial parameters, and also see what happens when we use an unknown name.

In [ ]:
# Step 3: Looking up and modifying parameter values from strings

print("Value of grid_points via bare name:")
print("  grid_points =", par.parval_from_str("grid_points"))

print("\nValue of grid_points via module::name syntax:")
print("  wave_tutorial::grid_points =", par.parval_from_str("wave_tutorial::grid_points"))

# Update a parameter and confirm that the stored value changes.
print("\nUpdating grid_points using set_parval_from_str...")
par.set_parval_from_str("grid_points", 256)

print("  Updated grid_points =", par.parval_from_str("grid_points"))
print("  Underlying stored value:", par.glb_params_dict["grid_points"].value)

# Try to look up a non existing parameter, catching the error so the notebook continues.
print("\nAttempting to look up a non existing parameter:")
try:
    par.parval_from_str("wave_tutorial::does_not_exist")
except ValueError as e:
    print("  Lookup failed with ValueError:")
    print("   ", e)

# Step 4: Exploring global parameter dictionaries and descriptions
### \[Back to [top](#Table-of-Contents)\]

All Python level parameters live in `par.glb_params_dict`, keyed by their names. This dictionary is helpful for:

* Discovering which parameters are active.
* Building basic help messages.
* Checking that modules registered the parameters you expected.

There is also `par.glb_extras_dict`, a free form dictionary of dictionaries that you can use to attach additional metadata.

In this step we:

* List all parameters belonging to the `"wave_tutorial"` module.
* Attach a small note into `glb_extras_dict` for later reference.

In [ ]:
# Step 4: Exploring global parameter dictionaries and descriptions

print("All Python level parameters belonging to the 'wave_tutorial' module:")
for name, p in par.glb_params_dict.items():
    if p.module == "wave_tutorial":
        print(f"  {p.module}::{p.name} = {p.value!r}  # {p.description}")

# Attach a small note to glb_extras_dict so that downstream scripts can see
# that this notebook configured the wave tutorial parameters.
if "wave_tutorial" not in par.glb_extras_dict:
    par.glb_extras_dict["wave_tutorial"] = {}

par.glb_extras_dict["wave_tutorial"]["notes"] = (
    "Parameters created in the wave tutorial params.ipynb example."
)

print("\nExtra metadata stored in glb_extras_dict for 'wave_tutorial':")
print(" ", par.glb_extras_dict["wave_tutorial"])

# Step 5: CodeParameters vs NRPyParameters: runtime tuning for generated code
### \[Back to [top](#Table-of-Contents)\]

Python level parameters are great for pure Python logic, but generated C code needs its own knob like objects. For this, `params.py` provides **CodeParameters**.

CodeParameters differ from Python level parameters in a few ways:

* Each CodeParameter has a C type string like `"REAL"`, `"int"`, `"REAL[3]"`, or `"bool"`.
* Each CodeParameter has an associated SymPy symbol that you can insert into analytic expressions.
* They are stored separately in `par.glb_code_params_dict`.

The main helper for single CodeParameters is:

```python
sym = par.register_CodeParameter(
    cparam_type="REAL",
    module="some_module",
    name="parameter_name",
    defaultvalue=...,
    assumption="Real",
    commondata=False,
    add_to_parfile=True,
    add_to_set_CodeParameters_h=True,
    description="..."
)
```

The return value `sym` is the SymPy symbol representing the parameter.

In this step we define a few CodeParameters that a simple 1D wave code might use: a CFL factor, a wave speed, and a boolean flag.

In [ ]:
# Step 5: CodeParameters vs NRPyParameters: runtime tuning for generated code

# For a clean example, we clear any previously registered CodeParameters.
par.glb_code_params_dict.clear()

# CFL factor controlling the stable time step.
cfl = par.register_CodeParameter(
    cparam_type="REAL",
    module="wave_tutorial_code",
    name="cfl_factor",
    defaultvalue=0.4,
    assumption="Real",
    commondata=True,
    add_to_parfile=True,
    add_to_set_CodeParameters_h=True,
    description="Courant factor controlling the stable time step.",
)

# Wave speed for the 1D wave equation.
wave_speed = par.register_CodeParameter(
    cparam_type="REAL",
    module="wave_tutorial_code",
    name="wave_speed",
    defaultvalue=1.0,
    assumption="RealPositive",
    commondata=True,
    add_to_parfile=True,
    add_to_set_CodeParameters_h=True,
    description="Characteristic wave speed c in the 1D wave equation.",
)

# Boolean flag enabling a more accurate (but more expensive) scheme.
use_high_order = par.register_CodeParameter(
    cparam_type="bool",
    module="wave_tutorial_code",
    name="use_high_order_stencils",
    defaultvalue=True,
    description="Enable higher order finite difference stencils.",
)

print("SymPy symbols returned by register_CodeParameter:")
print("  cfl_factor            ->", cfl)
print("  wave_speed            ->", wave_speed)
print("  use_high_order_stencils ->", use_high_order)

print("\nEntries stored in glb_code_params_dict:")
for name, p in par.glb_code_params_dict.items():
    print(
        f"  {name:24s}: cparam_type={p.cparam_type:5s}, "
        f"default={p.defaultvalue!r}, module={p.module}"
    )

# If we forget to supply a default while keeping add_to_parfile=True, the helper
# will complain. This is useful when wiring up many parameters at once.
print("\nAttempting to register a CodeParameter without a default value:")
try:
    par.register_CodeParameter(
        cparam_type="REAL",
        module="wave_tutorial_code",
        name="missing_default_example",
        # defaultvalue left at the sentinel to trigger the check
    )
except ValueError as e:
    print("  Caught ValueError:", e)

# Step 6: Registering multiple CodeParameters at once with register_CodeParameters()
### \[Back to [top](#Table-of-Contents)\]

It is common to have small groups of related CodeParameters with the same type, such as:

* A cluster of model coefficients.
* A pair of parameters controlling left and right boundary conditions.

Instead of calling `register_CodeParameter()` repeatedly, you can call:

```python
syms = par.register_CodeParameters(
    cparam_type="REAL",
    module="...",
    names=["a0", "a1", "b", "c"],
    defaultvalues=[...],
    descriptions=[...],
)
```

Helpful behavior:

* If `defaultvalues` is a single scalar, it is duplicated for each name.
* `descriptions` can be an empty string (no descriptions) or a list matching the length of `names`.

Here we create a small set of model coefficients and a pair of boundary condition selectors.

In [ ]:
# Step 6: Registering multiple CodeParameters at once with register_CodeParameters()

# Example 6a: Model coefficients with distinct defaults and descriptions.
coeff_a0, coeff_a1, coeff_b, coeff_c = par.register_CodeParameters(
    cparam_type="REAL",
    module="wave_tutorial_code",
    names=["coeff_a0", "coeff_a1", "coeff_b", "coeff_c"],
    defaultvalues=[1.0, 0.5, -0.25, 0.0],
    descriptions=[
        "Coefficient a0 in a toy model equation.",
        "Coefficient a1 in a toy model equation.",
        "Coefficient b in a toy model equation.",
        "Coefficient c in a toy model equation.",
    ],
)

print("Returned SymPy symbols for model coefficients:")
print(" ", coeff_a0, coeff_a1, coeff_b, coeff_c)

print("\nStored CodeParameters for model coefficients:")
for name in ["coeff_a0", "coeff_a1", "coeff_b", "coeff_c"]:
    param = par.glb_code_params_dict[name]
    print(
        f"  {name:10s}: defaultvalue={param.defaultvalue}, "
        f"cparam_type={param.cparam_type}, description='{param.description}'"
    )

# Example 6b: Boundary condition selectors sharing the same default.
bc_left, bc_right = par.register_CodeParameters(
    cparam_type="int",
    module="wave_tutorial_code",
    names=["bc_left_type", "bc_right_type"],
    defaultvalues=1,  # same default for both ends
    descriptions="",  # no extra descriptions
)

print("\nBroadcasted integer defaults for boundary condition types:")
for name in ["bc_left_type", "bc_right_type"]:
    param = par.glb_code_params_dict[name]
    print(f"  {name:12s}: defaultvalue={param.defaultvalue}, module={param.module}")

# It is useful to see what happens when descriptions have the wrong length.
print("\nAttempting to register parameters with mismatched descriptions:")
try:
    par.register_CodeParameters(
        cparam_type="bool",
        module="wave_tutorial_code",
        names=["flag_a", "flag_b"],
        defaultvalues=[True, False],
        descriptions=["Only one description here"],
    )
except ValueError as e:
    print("  Caught ValueError:", e)

# Step 7: Working with array valued CodeParameters
### \[Back to [top](#Table-of-Contents)\]

Some run time parameters are naturally small arrays, for example:

* A background state with three components.
* A short list of damping coefficients near a boundary.

Array valued CodeParameters are described using type strings such as `"REAL[3]"` or `"int[4]"`. They behave as follows:

* The type string encodes the base type and array size.
* The `defaultvalue` can be either:
  * A list of the correct length, or
  * A single scalar that will be repeated across the array.
* Internally, the `defaultvalue` is stored as a Python list.

When using `REAL[...]` or `int[...]` types, `add_to_set_CodeParameters_h` must be set to `False`, because these arrays are handled differently from simple scalar parameters.

In [ ]:
# Step 7: Working with array valued CodeParameters

# Example 7a: A three component background state.
background_state = par.register_CodeParameter(
    cparam_type="REAL[3]",
    module="wave_tutorial_code",
    name="background_state",
    defaultvalue=[0.0, 0.0, 0.0],
    assumption="Real",
    commondata=True,
    add_to_parfile=True,
    add_to_set_CodeParameters_h=False,
    description="Background state (e.g., fields) for the wave example.",
)

print("Array valued CodeParameter 'background_state':")
bg_param = par.glb_code_params_dict["background_state"]
print("  cparam_type  =", bg_param.cparam_type)
print("  defaultvalue =", bg_param.defaultvalue)

# Example 7b: Four damping coefficients using a single default value.
sponge_coeffs = par.register_CodeParameter(
    cparam_type="REAL[4]",
    module="wave_tutorial_code",
    name="sponge_coefficients",
    defaultvalue=0.05,  # will be broadcast to four entries
    commondata=True,
    add_to_parfile=True,
    add_to_set_CodeParameters_h=False,
    description="Damping coefficients used in a boundary sponge layer.",
)

print("\nArray valued CodeParameter 'sponge_coefficients':")
sp_param = par.glb_code_params_dict["sponge_coefficients"]
print("  cparam_type  =", sp_param.cparam_type)
print("  defaultvalue =", sp_param.defaultvalue)

# Show what happens if we forget to disable add_to_set_CodeParameters_h for arrays.
print("\nAttempting to register an array with add_to_set_CodeParameters_h=True:")
try:
    par.register_CodeParameter(
        cparam_type="REAL[2]",
        module="wave_tutorial_code",
        name="bad_array_example",
        defaultvalue=[0.0, 1.0],
        commondata=True,
        add_to_parfile=True,
        add_to_set_CodeParameters_h=True,  # not allowed for REAL[...] types
        description="This should trigger a ValueError.",
    )
except ValueError as e:
    print("  Caught ValueError:", e)

# Step 8: Adjusting CodeParameter defaults after registration
### \[Back to [top](#Table-of-Contents)\]

After CodeParameters are registered, you may want to tweak their default values based on higher level choices. Instead of re registering everything, you can use:

```python
par.adjust_CodeParam_default(CodeParameter_name, new_default, new_cparam_type="")
```

This helper supports:

* Scalar parameters, by passing the plain name `"cfl_factor"`.
* Array parameters, by indexing with a zero based index string such as `"sponge_coefficients[2]"`.
* Optional type changes: you can update the stored `cparam_type` by passing `new_cparam_type`.

This step demonstrates:

* Adjusting scalar defaults.
* Adjusting individual entries of array defaults.
* Changing a CodeParameter type string.
* The error raised for unknown names.

In [ ]:
# Step 8: Adjusting CodeParameter defaults after registration

print("Original defaults:")
print("  cfl_factor default         =", par.glb_code_params_dict["cfl_factor"].defaultvalue)
print("  wave_speed default         =", par.glb_code_params_dict["wave_speed"].defaultvalue)
print("  background_state default   =", par.glb_code_params_dict["background_state"].defaultvalue)
print("  sponge_coefficients default =", par.glb_code_params_dict["sponge_coefficients"].defaultvalue)

# Make the CFL factor slightly more restrictive.
par.adjust_CodeParam_default("cfl_factor", 0.3)

# Increase the wave speed and at the same time change the stored type string
# from "REAL" to "double". The parameter system does not interpret the type
# string, so this is purely bookkeeping, but it is sometimes useful.
par.adjust_CodeParam_default("wave_speed", 1.2, new_cparam_type="double")

# For array valued parameters, adjust selected entries using name[index] syntax.
par.adjust_CodeParam_default("background_state[0]", 0.1)
par.adjust_CodeParam_default("sponge_coefficients[3]", 0.1)

print("\nAfter adjustments:")
print("  cfl_factor default         =", par.glb_code_params_dict["cfl_factor"].defaultvalue)
print(
    "  wave_speed default         =", par.glb_code_params_dict["wave_speed"].defaultvalue,
    "; cparam_type =", par.glb_code_params_dict["wave_speed"].cparam_type,
)
print("  background_state default   =", par.glb_code_params_dict["background_state"].defaultvalue)
print("  sponge_coefficients default =", par.glb_code_params_dict["sponge_coefficients"].defaultvalue)

# Show the error when trying to adjust a non existent parameter.
print("\nAttempting to adjust a non existent CodeParameter:")
try:
    par.adjust_CodeParam_default("does_not_exist_here", 0.0)
except ValueError as e:
    print("  Caught ValueError:", e)

# Step 9: Using symbolic CodeParameters inside SymPy expressions and emitting C code
### \[Back to [top](#Table-of-Contents)\]

Each CodeParameter has an associated SymPy symbol, which lets you:

* Build analytic expressions that depend on tunable run time parameters.
* Do algebra, simplification, and substitution at the symbolic level.
* Turn the final result into C style code with SymPy's `ccode()` helper.

In this step we:

1. Add a grid spacing CodeParameter $dx$.
2. Build a simple expression for a time step $dt = \text{cfl\_factor} \cdot dx / \text{wave\_speed}$.
3. Substitute default values from `glb_code_params_dict` to estimate $dt$.
4. Emit C style code for $dt$.
5. Build a tiny model for an initial wave profile using other CodeParameters and emit its C code as well.

In [ ]:
# Step 9: Using symbolic CodeParameters inside SymPy expressions and emitting C code

# Grid spacing parameter dx, often used together with the CFL factor.
dx = par.register_CodeParameter(
    cparam_type="REAL",
    module="wave_tutorial_code",
    name="dx",
    defaultvalue=0.01,
    assumption="RealPositive",
    commondata=True,
    add_to_parfile=True,
    add_to_set_CodeParameters_h=True,
    description="Grid spacing dx for the wave tutorial.",
)

print("SymPy symbol for dx:", dx)

# Build dt = cfl_factor * dx / wave_speed using the symbolic handles.
dt = cfl * dx / wave_speed
print("\nSymbolic expression for dt = cfl_factor * dx / wave_speed:")
print("  dt =", dt)

# Substitute defaults from the global CodeParameter dictionary.
cfl_default = par.glb_code_params_dict["cfl_factor"].defaultvalue
dx_default = par.glb_code_params_dict["dx"].defaultvalue
wave_speed_default = par.glb_code_params_dict["wave_speed"].defaultvalue

dt_evaluated = dt.subs(
    {
        cfl: cfl_default,
        dx: dx_default,
        wave_speed: wave_speed_default,
    }
)

print("\nSubstituting default values from glb_code_params_dict:")
print(f"  cfl_factor default  = {cfl_default}")
print(f"  dx default          = {dx_default}")
print(f"  wave_speed default  = {wave_speed_default}")
print("  dt evaluated numeric =", float(dt_evaluated))

# Emit a C style expression for dt using SymPy's ccode.
print("\nC style representation of dt (using SymPy ccode):")
print(" ", sp.ccode(dt))

# As a second example, create a very simple initial data profile:
#    u(x) = coeff_a0 * sin(2*pi*x) + coeff_a1 * sin(4*pi*x)
# We will treat x as a C variable, while coeff_a0 and coeff_a1 are CodeParameters.
x = sp.Symbol("x", real=True)
pi = sp.pi

wave_profile = coeff_a0 * sp.sin(2 * pi * x) + coeff_a1 * sp.sin(4 * pi * x)
print("\nSymbolic initial data profile u(x):")
print("  u(x) =", wave_profile)

print("\nC style representation of u(x):")
print(" ", sp.ccode(wave_profile))